---
# 3 - Predicting performance
We know that Overall is the overall skill of a player.
The question is whether we can predict the Overall skill by looking at the other skills.

assumptions:
* the client prefers to measure a few skills rather than the entire suite.
* while all of these skill metrics are subjective, it is more objective to measure a player on specific skills,rather than to come up with an overall skill rating just from observation.

ideal deliverables:
* the client would like to know which of his players he should get rid of.
* the client would like to know which skills he should focus his players on to optimize overall skill.
* the client would like to know who are the most undervalued players, so he can recruit them.

In [ ]:
#remove the pluses and minuses from these numbers
import re

skill_columns = ['Acceleration',
       'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure',
       'Crossing', 'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy',
       'GK diving', 'GK handling', 'GK kicking', 'GK positioning',
       'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping',
       'Long passing', 'Long shots', 'Marking', 'Penalties', 'Positioning',
       'Reactions', 'Short passing', 'Shot power', 'Sliding tackle',
       'Sprint speed', 'Stamina', 'Standing tackle', 'Strength', 'Vision',
       'Volleys']

def stripValue(value):
    if isinstance(value, str):
        return int(re.findall('\d+',value)[0])
    else:
        return None

#is this the right way to do this ?why am I getting a warning?
for column in skill_columns:
    player_stats[column] = player_stats[column].apply(stripValue)

In [ ]:
player_stats.head()

In [ ]:
player_stats.dropna(inplace=True)

In [ ]:
for column in skill_columns:
    sns.distplot(player_stats[column],bins=50)

In [ ]:
#I want to analyze the connections between these different features, 
#however this particular notebook doesn't have the tools to do so .
#it is tempting to go ahead and find those tools,but I should stay on track 

#I want to know the correlation between each of these skills and overall skill
#I want to do a principal component analysis to reduce these skills to the single most important ones
#I want to see if a particular skill is more important than another
#I want to discover what is the lowest mix of skills that optimizes for the value of the player
#I want to see which minimal features do a good job of predicting overall skill



## Simple Linear Regression: 2 independent variables

the rest of this notebook is linear regression
I need to pick the top 3 skills that are most highly correlated with overall skill and value
then I need to do a linear regression and assess how strong that is right paren

In [ ]:
#correlation
correlation_matrix = player_stats.corr()
fig, ax = plt.subplots(figsize=(20,20)) 
sns.heatmap(correlation_matrix,annot=False,cmap='coolwarm', linewidths=.5, ax=ax)
#
#the strongest skill correlation 2 Overall is Reactions
#I don't think this is a skill.
#first I'll try to make a linear regression between the top 3 skills and overall
#then I will try to take all skills and see if that gives me a better model

In [ ]:
#skills with the highest correlation
correlation_overall = correlation_matrix['Overall'].sort_values(ascending=False)

correlation_overall.head(15)
#I'm only going to use reactions and composure in this first model.
#the other values are not skills,and of the other skills are too lowly correlated

In [ ]:
#define the training and testing data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

y = player_stats['Overall']
X = player_stats[['Reactions','Composure']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

lm = LinearRegression()
lm.fit(X_train,y_train)

coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coefficient']
coeffecients

In [ ]:
#compare predicted to actual values
predictions = lm.predict(X_test)

plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

## Evaluating the Model

evaluate  model performance by calculating the residual sum of squares and the explained variance score (R^2).

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#make sure residuals are normally distributed and not too big
sns.distplot((y_test-predictions),bins=50);

# first conclusion
the only highly correlated skill to overall is reactions.I created a linear regression model
that can predict overall fairly well.I'm not very excited about this though.
there is more to the story.I wouldn't tell management to look for players with good reactions
in order to build a good team.

I am going to build another model,and I will leave out reactions.let's see how good it can be.
it will only take into account real,more measurable skills.

## Simple Linear Regression: low correlation features
in this model, I am only going to use features with low correlation to see how good of a predictor I can create.

In [ ]:
#skills with the highest correlation
correlation_overall = correlation_matrix['Overall'].sort_values(ascending=False)

#exclude features with the highest correlations,those above 0.53
low_correlated_skills  = correlation_overall[correlation_overall < 0.53].index.values
low_correlated_skills

In [ ]:
#define the training and testing data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

y = player_stats['Overall']
X = player_stats[low_correlated_skills]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

lm = LinearRegression()
lm.fit(X_train,y_train)

coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coefficient']
coeffecients

In [ ]:
#compare predicted to actual values
predictions = lm.predict(X_test)

plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

## Evaluating the Model

evaluate  model performance by calculating the residual sum of squares and the explained variance score (R^2).

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#make sure residuals are normally distributed and not too big
sns.distplot((y_test-predictions),bins=50);

# second conclusion
in this second model I excluded the most highly correlated features,those being reactions and composure.
I wanted to see if I could create a predictive model using very lowly correlated features.my assumption was that the higher the correlation,the more likely that feature predicts the y value.perhaps this is true,and there is another truth in here.the other truth is that a large group of lowly correlated features can also give a good predictive model.

now my goal is to create the most predictive model.I will do this by including the highly correlated features as well as the lowly correlated features into my model.

## Simple Linear Regression: all features


In [ ]:
#define the training and testing data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

y = player_stats['Overall']
X = player_stats[skill_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

lm = LinearRegression()
lm.fit(X_train,y_train)

coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coefficient']
coeffecients.sort_values(by='Coefficient',ascending=False)

even though composure is more highly correlated then ball control, here ball control has a higher coefficient then  composure. I must have a wrong assumption about how this math works. something I can try to do is to see the correlation between coefficients and correlation values.

In [ ]:
#compare predicted to actual values
predictions = lm.predict(X_test)

plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

## Evaluating the Model

evaluate  model performance by calculating the residual sum of squares and the explained variance score (R^2).

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#make sure residuals are normally distributed and not too big
sns.distplot((y_test-predictions),bins=50);

## third conclusion
it appears that the best model is when you use the most features.

* highly correlated features RMSE: 3.31230582722
* only low correlated future RMSE: 3.25479269275
* all features RMSE: 2.62305737694

## critique

1. your initial goal was to study a player's performance
    * you didn't formulate clear questions: 
        * which metric is the best representation for performance? _overall_
        * which smaller metric is the best predictor for performance?
        * which set of 5 smaller metrics are the best predictors for performance?
    * you didn't make recommendations to your client:
        * should players focus on their best skills or the worst skills to get better performance?
        * which players have the best wage to value ratio?
        * which players have the best wage to overall ratio?
        * who are the weakest players on any given team?
        
2. your secondary goal was to study a team's performance
    * you don't have data on teams winning
    * you didn't study the connection between individual players and team skill
        * what is the best player mix for a good team?
        * what is the best way to predict a team skill based on individual player features?

3. your third goal was to make financial recommendations 
    * which teams have the best wage to value ratios?
    * which teams have the best wage to overall ratios?
    
---

1. each section needs an explicit goal, with steps laid out
2. each section needs an explicit conclusion, that cites the values discovered
3. whenever you learn something from a calculation, you should write that down
4. make your variable names more clear